# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("cities.csv") 
cities_df

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Punta Arenas,-53.15,-70.92,33.80,55,40,6.93,CL,1598832188
1,1,Caravelas,-17.71,-39.25,70.99,91,0,4.92,BR,1598832223
2,2,Hasaki,35.73,140.83,80.01,88,75,10.29,JP,1598832192
3,3,Karratha,-20.74,116.85,80.33,28,3,6.89,AU,1598832252
4,4,Barrow,71.29,-156.79,39.20,86,90,17.22,US,1598832202
...,...,...,...,...,...,...,...,...,...,...
570,570,Thinadhoo,0.53,72.93,82.65,71,99,7.90,MV,1598832555
571,571,St. George's,12.06,-61.75,80.60,94,75,4.25,GD,1598832555
572,572,Wembley,-31.93,115.80,53.01,70,0,5.82,AU,1598832117
573,573,Tocopilla,-22.09,-70.20,59.58,69,100,5.48,CL,1598832556


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
center_coordinates = (0, 0)
fig = gmaps.figure(center=center_coordinates, zoom_level=2)
cities_locations = cities_df[["lat", "lng"]].astype(float)

pct_humidity = cities_df["humidity"].astype(float)

In [5]:
heat_layer = gmaps.heatmap_layer(cities_locations, weights=pct_humidity) 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = cities_df.loc[(cities_df["max_temp"] <= 72) &
                         (cities_df["max_temp"] >= 68) &
                         (cities_df["humidity"] < 60) &
                         (cities_df["cloudiness"] < 30)]
hotel_df = hotel_df.dropna(axis = 0, how ='any') 
hotel_df

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
78,78,Pacific Grove,36.62,-121.92,72.00,56,1,8.05,US,1598832275
92,92,College,64.86,-147.80,68.00,39,20,11.41,US,1598832238
159,159,Cabra,37.47,-4.44,69.01,33,0,3.06,ES,1598832209
199,199,Baijiantan,45.63,85.18,69.75,33,0,3.67,CN,1598832426
262,262,Yangi Marg`ilon,40.43,71.72,69.80,52,0,6.93,UZ,1598832235
297,297,Fortuna,40.60,-124.16,70.00,48,20,13.87,US,1598832239
320,320,Abrau-Dyurso,44.69,37.59,71.60,40,0,13.42,RU,1598832480
486,486,Kashan,33.98,51.44,70.18,26,0,5.79,IR,1598832531
495,495,Nelson Bay,-32.72,152.15,69.01,49,5,23.04,AU,1598832260
569,569,Moranbah,-22.00,148.05,70.83,46,0,12.12,AU,1598832288


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_name = []
for i in range(len(hotel_df)):
    # geocoordinates
    target_coordinates = f"{hotel_df.iloc[i, 2]}, {hotel_df.iloc[i, 3]}"
    target_radius = 5000
    target_type = "hotel"
    # print(hotel_df.iloc[1, 1])

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    # print(json.dumps(places_data, indent=4, sort_keys=True))
    # print(places_data["results"][0]["name"])
    hotel_name.append(places_data["results"][0]["name"])
# print(hotel_name)
hotel_df['Hotel Name'] = hotel_name
hotel_df.rename(columns = {'lat':'Lat', 'lng':'Lng', 'city':'City', 'country':'Country'}, inplace = True)
hotel_df['Lat'] = hotel_df['Lat'].astype(float)
hotel_df['Lng'] = hotel_df['Lng'].astype(float)
hotel_df

,Unnamed: 0,City,Lat,Lng,max_temp,humidity,cloudiness,wind_speed,Country,date,Hotel Name
78,78,Pacific Grove,36.62,-121.92,72.00,56,1,8.05,US,1598832275,Monterey
92,92,College,64.86,-147.80,68.00,39,20,11.41,US,1598832238,Fairbanks
159,159,Cabra,37.47,-4.44,69.01,33,0,3.06,ES,1598832209,Cabra
199,199,Baijiantan,45.63,85.18,69.75,33,0,3.67,CN,1598832426,Karamay
262,262,Yangi Marg`ilon,40.43,71.72,69.80,52,0,6.93,UZ,1598832235,Fergana
297,297,Fortuna,40.60,-124.16,70.00,48,20,13.87,US,1598832239,Fortuna
320,320,Abrau-Dyurso,44.69,37.59,71.60,40,0,13.42,RU,1598832480,Primorskiy Rayon
486,486,Kashan,33.98,51.44,70.18,26,0,5.79,IR,1598832531,Kashan
495,495,Nelson Bay,-32.72,152.15,69.01,49,5,23.04,AU,1598832260,Nelson Bay - Corlette
569,569,Moranbah,-22.00,148.05,70.83,46,0,12.12,AU,1598832288,Moranbah


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
hotel_info = []
for i in range(len(hotel_df)):
    hotel_info.append(f"Hotel Name: {hotel_df.iloc[i,10]}, City: {hotel_df.iloc[i,1]}, Country: {hotel_df.iloc[i,8]}")

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))